In [1]:
import math
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as autograd
import torch.optim as optim
import numpy as np
from torch.autograd import Variable
import import_ipynb
#from model import GZSL
import model
import argparse


importing Jupyter notebook from model.ipynb


In [2]:
learning_rate = 0.0009
num_epochs = 5000
vae_epochs = 2000
kl_weight_max = 1
lambda_r = 0.1
lambda_e = 0.1
lambda_reg = 0.1
lambda_c = 0.1
batch_size = 100
   

In [3]:
import numpy as np

trainData = np.load(open('trainData' , 'r'))
trainLabels = np.load(open('trainLabels' , 'r'))
trainLabelVectors = np.load(open('trainAttributes' , 'r'))



path = '/home/haindavi/cv/proj/data/Animals_with_Attributes2/'

fp_attr = open(path + 'predicate-matrix-continuous.txt' ,'r')
cls_attr   =[[float(a) for a in x.strip().split(' ') if a] for x in fp_attr.readlines()]
cls_attr = np.array(cls_attr)

def load_batch(data,batch_size):
    img, labels = data
    size = img.shape[0]
    idx = np.random.choice(size, batch_size)
    batch_data = img[idx]
    batch_labels = labels[idx]
    
    batch_data = torch.from_numpy(np.array(batch_data))
    batch_data = batch_data.type(torch.FloatTensor)
    batch_labels = torch.from_numpy(np.array(batch_labels))
    batch_labels = batch_labels.type(torch.FloatTensor)
    
    return batch_data, batch_labels


In [4]:
def sample_c(batch_size):
    idx = np.random.choice(cls_attr.shape[0], batch_size)
    cls = cls_attr[idx]
    cls = torch.from_numpy(np.array(cls))
    cls = cls.type(torch.FloatTensor)
    return cls


In [5]:
def train_vae(model):
    j = 0
    for epoch in range(vae_epochs):
        seen_x, seen_class_labels = load_batch((trainData,trainLabelVectors),batch_size)

        recon_loss, kl_loss = model.forward(seen_x, seen_class_labels)
        loss_vae = recon_loss + kl_weight_max * kl_loss

        loss_vae.backward()
        grad_norm = torch.nn.utils.clip_grad_norm(model.vae_params, 5)
        vae_optim.step()
        vae_optim.zero_grad()
        j = j+1
        if (j + 1) % 10 == 0:
            print ('Epoch [{}/{}], Step [{}], Loss VAE: {:.4f}'.format(epoch+1, num_epochs, j + 1, loss_vae.item()))

    return model

In [6]:
def save_model(model):
    if not os.path.exists('models/'):
        os.makedirs('models/')
    torch.save(model.state_dict(), 'models/gzsl.bin')

In [7]:

feat_dim = 2048 
cls_dim = 85
enc_hdim1 = 512
enc_hdim2 = 512 
zdim = 50 
dec_hdim1 = 512 
disc_hdim1 = 512

model = model.GZSL(feat_dim, cls_dim, enc_hdim1, enc_hdim2, zdim, dec_hdim1, disc_hdim1
 )

vae_optim = optim.Adam(model.vae_params, lr = learning_rate)
discrim_optim = optim.Adam(model.disc_params, lr = learning_rate)

i = 0
def train_gzsl(model):
    for epoch in range(num_epochs):

        seen_x, seen_class_labels = load_batch((trainData,trainLabelVectors),batch_size)
            
        #train discriminator
        attr = sample_c(batch_size)
        generated_x, generated_class  = model.generate_images(batch_size, attr)

        y_disc_real = model.forward_discriminator(seen_x)
        y_disc_fake = model.forward_discriminator(generated_x)

        loss_s = F.mse_loss(y_disc_real, seen_class_labels, size_average= True)
        loss_u = F.mse_loss(y_disc_fake, generated_class, size_average = True)

        loss_discrim = loss_s + lambda_r*loss_u

        loss_discrim.backward()
        grad_norm = torch.nn.utils.clip_grad_norm(model.disc_params, 1)
        discrim_optim.step()
        discrim_optim.zero_grad()
            
        #train generator and encoder     
        recon_loss, kl_loss = model.forward(seen_x, seen_class_labels, use_c_prior=True)
        loss_vae = recon_loss + kl_weight_max * kl_loss

        attr1 = sample_c(batch_size)
        generated_x1, generated_class1  = model.generate_images(batch_size, attr1)

        y_disc_fake1 = model.forward_discriminator(generated_x1)        

        loss_c = F.mse_loss(y_disc_fake1, generated_class1)

        loss_reg =  F.mse_loss(generated_x1, seen_x)

        x_hat_in = torch.cat((generated_x1,generated_class1),1)
        z_hat_mu, z_hat_log_var  = model.forward_encoder(x_hat_in)

        loss_e = torch.mean(0.5 * torch.sum(torch.exp(z_hat_log_var) + z_hat_mu**2 - 1 - z_hat_log_var, 1))
            

        loss_vae_pair = loss_vae + lambda_c*loss_c + lambda_reg*loss_reg + lambda_e*loss_e

        loss_vae_pair.backward()
        grad_norm = torch.nn.utils.clip_grad_norm(model.vae_params, 1)
        vae_optim.step()
        vae_optim.zero_grad()
        global i
        i= i+1
        print(loss_u.item(), loss_s.item())
            
        if (i + 1) % 100 == 0:
            print(loss_u.item(), loss_s.item())
            save_model(model)
            print ('Epoch [{}/{}], Step [{}], Loss Discriminator: {:.4f}, Loss VAE pair: {:.4f}, Loss VAE: {:.4f}, Loss c: {:.4f}, Loss reg: {:.4f}, Loss e: {:.4f}'.format(epoch+1, num_epochs, i + 1, loss_discrim.item(), loss_vae_pair.item(), loss_vae.item(), loss_c.item(), loss_reg.item(), loss_e.item()))

    return model       

In [8]:
#model = GZSL()
#model.load_state_dict(torch.load('modinels/vae.bin'))
model = train_vae(model)
#model.load_state_dict(torch.load('models/gzsl.bin'))

print("training GZSL nowww...............................")
model = train_gzsl(model)
save_model(model)

/home/haindavi/anaconda2/lib/python2.7/site-packages/torch/nn/functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='elementwise_mean' instead.
  warnings.warn(warning.format(ret))
/home/haindavi/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:14: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  


Epoch [9/5000], Step [10], Loss VAE: 2.8091
Epoch [19/5000], Step [20], Loss VAE: 0.7776
Epoch [29/5000], Step [30], Loss VAE: 0.5443
Epoch [39/5000], Step [40], Loss VAE: 0.4366
Epoch [49/5000], Step [50], Loss VAE: 0.4106
Epoch [59/5000], Step [60], Loss VAE: 0.3966
Epoch [69/5000], Step [70], Loss VAE: 0.3883
Epoch [79/5000], Step [80], Loss VAE: 0.3511
Epoch [89/5000], Step [90], Loss VAE: 0.3482
Epoch [99/5000], Step [100], Loss VAE: 0.3181
Epoch [109/5000], Step [110], Loss VAE: 0.3786
Epoch [119/5000], Step [120], Loss VAE: 0.3458
Epoch [129/5000], Step [130], Loss VAE: 0.3653
Epoch [139/5000], Step [140], Loss VAE: 0.3221
Epoch [149/5000], Step [150], Loss VAE: 0.3542
Epoch [159/5000], Step [160], Loss VAE: 0.3858
Epoch [169/5000], Step [170], Loss VAE: 0.3493
Epoch [179/5000], Step [180], Loss VAE: 0.3397
Epoch [189/5000], Step [190], Loss VAE: 0.3347
Epoch [199/5000], Step [200], Loss VAE: 0.3349
Epoch [209/5000], Step [210], Loss VAE: 0.3372
Epoch [219/5000], Step [220], Los

Epoch [1729/5000], Step [1730], Loss VAE: 0.3812
Epoch [1739/5000], Step [1740], Loss VAE: 0.3360
Epoch [1749/5000], Step [1750], Loss VAE: 0.3581
Epoch [1759/5000], Step [1760], Loss VAE: 0.3624
Epoch [1769/5000], Step [1770], Loss VAE: 0.3087
Epoch [1779/5000], Step [1780], Loss VAE: 0.3502
Epoch [1789/5000], Step [1790], Loss VAE: 0.3147
Epoch [1799/5000], Step [1800], Loss VAE: 0.3317
Epoch [1809/5000], Step [1810], Loss VAE: 0.3557
Epoch [1819/5000], Step [1820], Loss VAE: 0.3668
Epoch [1829/5000], Step [1830], Loss VAE: 0.3209
Epoch [1839/5000], Step [1840], Loss VAE: 0.3289
Epoch [1849/5000], Step [1850], Loss VAE: 0.3289
Epoch [1859/5000], Step [1860], Loss VAE: 0.3579
Epoch [1869/5000], Step [1870], Loss VAE: 0.3465
Epoch [1879/5000], Step [1880], Loss VAE: 0.3240
Epoch [1889/5000], Step [1890], Loss VAE: 0.3259
Epoch [1899/5000], Step [1900], Loss VAE: 0.3031
Epoch [1909/5000], Step [1910], Loss VAE: 0.3170
Epoch [1919/5000], Step [1920], Loss VAE: 0.3318
Epoch [1929/5000], S

/home/haindavi/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:41: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
/home/haindavi/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:72: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.


(1053.102294921875, 1047.69677734375)
(982.4219970703125, 1057.6253662109375)
(930.0155639648438, 990.9208984375)
(861.3341674804688, 943.9044189453125)
(668.3759155273438, 885.745849609375)
(554.3480224609375, 811.4168701171875)
(630.8765258789062, 773.2305297851562)
(891.9375610351562, 701.2597045898438)
(1112.002685546875, 661.0555419921875)
(1057.015625, 589.887451171875)
(853.7091064453125, 592.8386840820312)
(604.6497192382812, 587.4716186523438)
(450.4578857421875, 573.144775390625)
(442.2737731933594, 552.3543701171875)
(494.85736083984375, 525.9727783203125)
(481.50628662109375, 528.3914794921875)
(494.8426818847656, 481.6197509765625)
(458.0469665527344, 481.6230163574219)
(419.8770446777344, 457.83245849609375)
(459.4588317871094, 441.65679931640625)
(500.5036315917969, 469.0005187988281)
(425.4356384277344, 446.39373779296875)
(382.2229309082031, 421.9497985839844)
(421.5591735839844, 448.7159118652344)
(436.150146484375, 461.1847229003906)
(411.4109802246094, 412.494598388

...

(1.1766667366027832, 16.953487396240234)
(1.1220040321350098, 15.82341194152832)
(1.0370135307312012, 15.440983772277832)
(1.2653899192810059, 17.035614013671875)
(1.1752885580062866, 13.93612003326416)
(1.1357778310775757, 11.551305770874023)
(1.2941739559173584, 14.136022567749023)
(1.0699824094772339, 12.823107719421387)
(1.0673048496246338, 13.954192161560059)
(0.8832271695137024, 12.117168426513672)
(0.9495347738265991, 15.37579345703125)
(0.9801463484764099, 13.285246849060059)
(1.0685796737670898, 15.471355438232422)
(0.9271585941314697, 12.25516128540039)
(1.029029130935669, 15.84558391571045)
(1.1870163679122925, 12.150014877319336)
(1.1894164085388184, 18.449350357055664)
(1.5795458555221558, 21.604867935180664)
(1.2212659120559692, 13.37332820892334)
(1.6389156579971313, 15.081883430480957)
(1.5069831609725952, 14.081042289733887)
(1.2358582019805908, 19.998319625854492)
(1.0091050863265991, 12.881843566894531)
(1.8869730234146118, 14.48554801940918)
(2.258695363998413, 16.2

(1.343206524848938, 13.693893432617188)
(1.1171292066574097, 17.422588348388672)
(1.1675666570663452, 14.901439666748047)
(1.0782655477523804, 15.816489219665527)
(1.2359338998794556, 14.793331146240234)
(1.0442956686019897, 14.440340995788574)
(1.121751070022583, 17.60523223876953)
(1.1592696905136108, 15.712459564208984)
(1.1989073753356934, 10.500669479370117)
(1.0913341045379639, 15.962142944335938)
(0.9788152575492859, 16.4986572265625)
(0.9957773685455322, 14.123064041137695)
(0.9895946979522705, 11.66430950164795)
(1.0111953020095825, 13.339083671569824)
(1.1640269756317139, 16.87844467163086)
(1.141814112663269, 18.2264347076416)
(1.0279781818389893, 11.788237571716309)
(1.1109833717346191, 19.844186782836914)
(1.2393897771835327, 12.831425666809082)
(1.124314308166504, 13.774319648742676)
(1.2946501970291138, 15.241870880126953)
(1.3720937967300415, 14.010697364807129)
(1.23878812789917, 13.256546974182129)
(1.0159634351730347, 11.16617202758789)
(0.8594105839729309, 11.303174

(1.0369166135787964, 14.804207801818848)
(1.118753433227539, 13.49409008026123)
(1.0763955116271973, 14.307239532470703)
(1.0633312463760376, 14.663193702697754)
(0.9188794493675232, 11.609785079956055)
(1.0399904251098633, 11.463719367980957)
(1.064362645149231, 14.104318618774414)
(0.9932648539543152, 12.992342948913574)
(0.9582072496414185, 16.648900985717773)
(0.9189425110816956, 18.157258987426758)
(0.8243164420127869, 11.196751594543457)
(0.9731912612915039, 15.409744262695312)
(1.0183825492858887, 16.806766510009766)
(1.23124361038208, 13.488430976867676)
(1.2182272672653198, 11.009112358093262)
(1.5214941501617432, 12.586949348449707)
(1.3118822574615479, 13.518671035766602)
(1.275015115737915, 13.482359886169434)
(1.3344331979751587, 16.803695678710938)
(0.9174143075942993, 12.783391952514648)
(1.168746829032898, 13.760272979736328)
(1.586475133895874, 11.353440284729004)
(1.508683204650879, 14.885560035705566)
(1.3046185970306396, 14.322234153747559)
(1.3089416027069092, 13.5

(1.205062985420227, 11.85647201538086)
(1.0788254737854004, 14.866354942321777)
(1.1317464113235474, 13.851204872131348)
(1.0396230220794678, 10.641894340515137)
(0.9195045232772827, 15.1138277053833)
(0.967465877532959, 12.011225700378418)
(0.8374311923980713, 15.46429443359375)
(0.9079886674880981, 12.309659004211426)
(0.704003632068634, 10.650067329406738)
(0.8415737152099609, 11.933951377868652)
(1.263879656791687, 12.126866340637207)
(1.0167735815048218, 15.463969230651855)
(0.8948130011558533, 11.013849258422852)
(1.0526021718978882, 12.665903091430664)
(1.2602620124816895, 11.900789260864258)
(1.0617966651916504, 13.620572090148926)
(1.4175114631652832, 14.806824684143066)
(1.5917540788650513, 15.500252723693848)
(1.3688409328460693, 14.335103988647461)
(1.1189111471176147, 9.069519996643066)
(1.0607306957244873, 15.64616584777832)
(1.3789299726486206, 12.3018159866333)
(1.27765691280365, 16.66712188720703)
(1.0259408950805664, 10.753496170043945)
(1.3004783391952515, 11.2973890

(1.414587378501892, 14.268976211547852)
(1.0898516178131104, 14.489758491516113)
(0.9306600093841553, 9.999908447265625)
(0.8254318237304688, 14.844465255737305)
(0.811984121799469, 14.625324249267578)
(0.904016375541687, 13.13074016571045)
(0.9856902956962585, 14.917269706726074)
(0.9752710461616516, 11.589776992797852)
(0.9486129879951477, 17.88677406311035)
(1.063332200050354, 11.554325103759766)
(0.9842699766159058, 11.481353759765625)
(1.230102300643921, 13.657370567321777)
(1.2535878419876099, 10.757954597473145)
(1.2645783424377441, 10.948391914367676)
(0.9756599068641663, 14.549765586853027)
(1.051939606666565, 14.805392265319824)
(0.8769513964653015, 8.641326904296875)
(1.2160327434539795, 16.01091766357422)
(1.0629769563674927, 11.213624954223633)
(1.0976805686950684, 11.432438850402832)
(1.0515505075454712, 9.620180130004883)
(1.0787339210510254, 11.487367630004883)
(1.0394331216812134, 13.493531227111816)
(1.045683741569519, 10.712695121765137)
(1.5809237957000732, 15.44188